# Project Data Wrangling 

### by Thioro Fall

# Wrangling data
+ Gathering data
+ Assessing data
+ Cleaning data

# Introduction
In this project, we will be wrangling and analyzing data from a twitter account WeRateDogs that rates people's dogs with a humorous comment about the dog.
The WeRateDogs Twitter archive data contains basic tweet data for all 5000+ of their tweets.

# Gathering data

In [2]:
# Import all the packages we need 
import numpy as np
import pandas as pd
import tweepy
import requests
import json
import time
import os
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
# Import the Twitter archive CSV file into a DataFrame
df_twitter_archive = pd.read_csv("twitter_archive_enhanced.csv")

In [4]:
df_twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [5]:
# Create a copy of df_twitter_archive
df_twitter = df_twitter_archive.copy()

In [6]:
# Download tweet image predictions TSV using the Requests library and write it to image_predictions.tsv
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open('image_predictions.tsv', mode='wb') as file:
    file.write(response.content)
    
# Import the tweet image predictions TSV file into a DataFrame
df_image = pd.read_csv('image_predictions.tsv', sep='\t')

In [7]:
df_image.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [8]:
# create a copy of df-img
df_img = df_image.copy()

Next, we need to  extract twitter data using API keys. This cell ONLY needs to be ran ONCE!!!!<br>
We will then download and save the JSON data as a text file from Udacity.

In [26]:
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = df_twitter.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

1: 892420643555336193
Fail
2: 892177421306343426
Fail
3: 891815181378084864
Fail
4: 891689557279858688
Fail
5: 891327558926688256
Fail
6: 891087950875897856
Fail
7: 890971913173991426
Fail
8: 890729181411237888
Fail
9: 890609185150312448
Fail
10: 890240255349198849
Fail
11: 890006608113172480
Fail
12: 889880896479866881
Fail
13: 889665388333682689
Fail
14: 889638837579907072
Fail
15: 889531135344209921
Fail
16: 889278841981685760
Fail
17: 888917238123831296
Fail
18: 888804989199671297
Fail
19: 888554962724278272
Fail
20: 888202515573088257
Fail
21: 888078434458587136
Fail
22: 887705289381826560
Fail
23: 887517139158093824
Fail
24: 887473957103951883
Fail
25: 887343217045368832
Fail
26: 887101392804085760
Fail
27: 886983233522544640
Fail
28: 886736880519319552
Fail
29: 886680336477933568
Fail
30: 886366144734445568
Fail
31: 886267009285017600
Fail
32: 886258384151887873
Fail
33: 886054160059072513
Fail
34: 885984800019947520
Fail
35: 885528943205470208
Fail
36: 885518971528720385
Fail
3

Fail
288: 838561493054533637
Fail
289: 838476387338051585
Fail
290: 838201503651401729
Fail
291: 838150277551247360
Fail
292: 838085839343206401
Fail
293: 838083903487373313
Fail
294: 837820167694528512
Fail
295: 837482249356513284
Fail
296: 837471256429613056
Fail
297: 837366284874571778
Fail
298: 837110210464448512
Fail
299: 837012587749474308
Fail
300: 836989968035819520
Fail
301: 836753516572119041
Fail
302: 836677758902222849
Fail
303: 836648853927522308
Fail
304: 836397794269200385
Fail
305: 836380477523124226
Fail
306: 836260088725786625
Fail
307: 836001077879255040
Fail
308: 835685285446955009
Fail
309: 835574547218894849
Fail
310: 835536468978302976
Fail
311: 835309094223372289
Fail
312: 835297930240217089
Fail
313: 835264098648616962
Fail
314: 835246439529840640
Fail
315: 835172783151792128
Fail
316: 835152434251116546
Fail
317: 834931633769889797
Fail
318: 834786237630337024
Fail
319: 834574053763584002
Fail
320: 834477809192075265
Fail
321: 834458053273591808
Fail
322: 8342

Fail
572: 801538201127157760
Fail
573: 801285448605831168
Fail
574: 801167903437357056
Fail
575: 801127390143516673
Fail
576: 801115127852503040
Fail
577: 800859414831898624
Fail
578: 800855607700029440
Fail
579: 800751577355128832
Fail
580: 800513324630806528
Fail
581: 800459316964663297
Fail
582: 800443802682937345
Fail
583: 800388270626521089
Fail
584: 800188575492947969
Fail
585: 800141422401830912
Fail
586: 800018252395122689
Fail
587: 799774291445383169
Fail
588: 799757965289017345
Fail
589: 799422933579902976
Fail
590: 799308762079035393
Fail
591: 799297110730567681
Fail
592: 799063482566066176
Fail
593: 798933969379225600
Fail
594: 798925684722855936
Fail
595: 798705661114773508
Fail
596: 798701998996647937
Fail
597: 798697898615730177
Fail
598: 798694562394996736
Fail
599: 798686750113755136
Fail
600: 798682547630837760
Fail
601: 798673117451325440
Fail
602: 798665375516884993
Fail
603: 798644042770751489
Fail
604: 798628517273620480
Fail
605: 798585098161549313
Fail
606: 7985

Fail
856: 764857477905154048
Fail
857: 764259802650378240
Fail
858: 763956972077010945
Fail
859: 763837565564780549
Fail
860: 763183847194451968
Fail
861: 763167063695355904
Fail
862: 763103485927849985
Fail
863: 762699858130116608
Fail
864: 762471784394268675
Fail
865: 762464539388485633
Fail
866: 762316489655476224
Fail
867: 762035686371364864
Fail
868: 761976711479193600
Fail
869: 761750502866649088
Fail
870: 761745352076779520
Fail
871: 761672994376806400
Fail
872: 761599872357261312
Fail
873: 761371037149827077
Fail
874: 761334018830917632
Fail
875: 761292947749015552
Fail
876: 761227390836215808
Fail
877: 761004547850530816
Fail
878: 760893934457552897
Fail
879: 760656994973933572
Fail
880: 760641137271070720
Fail
881: 760539183865880579
Fail
882: 760521673607086080
Fail
883: 760290219849637889
Fail
884: 760252756032651264
Fail
885: 760190180481531904
Fail
886: 760153949710192640
Fail
887: 759943073749200896
Fail
888: 759923798737051648
Fail
889: 759846353224826880
Fail
890: 7597

Fail
1134: 728751179681943552
Fail
1135: 728653952833728512
Fail
1136: 728409960103686147
Fail
1137: 728387165835677696
Fail
1138: 728046963732717569
Fail
1139: 728035342121635841
Fail
1140: 728015554473250816
Fail
1141: 727685679342333952
Fail
1142: 727644517743104000
Fail
1143: 727524757080539137
Fail
1144: 727314416056803329
Fail
1145: 727286334147182592
Fail
1146: 727175381690781696
Fail
1147: 727155742655025152
Fail
1148: 726935089318363137
Fail
1149: 726887082820554753
Fail
1150: 726828223124897792
Fail
1151: 726224900189511680
Fail
1152: 725842289046749185
Fail
1153: 725786712245440512
Fail
1154: 725729321944506368
Fail
1155: 725458796924002305
Fail
1156: 724983749226668032
Fail
1157: 724771698126512129
Fail
1158: 724405726123311104
Fail
1159: 724049859469295616
Fail
1160: 724046343203856385
Fail
1161: 724004602748780546
Fail
1162: 723912936180330496
Fail
1163: 723688335806480385
Fail
1164: 723673163800948736
Fail
1165: 723179728551723008
Fail
1166: 722974582966214656
Fail
1167:

Fail
1408: 699079609774645248
Fail
1409: 699072405256409088
Fail
1410: 699060279947165696
Fail
1411: 699036661657767936
Fail
1412: 698989035503689728
Fail
1413: 698953797952008193
Fail
1414: 698907974262222848
Fail
1415: 698710712454139905
Fail
1416: 698703483621523456
Fail
1417: 698635131305795584
Fail
1418: 698549713696649216
Fail
1419: 698355670425473025
Fail
1420: 698342080612007937
Fail
1421: 698262614669991936
Fail
1422: 698195409219559425
Fail
1423: 698178924120031232
Fail
1424: 697995514407682048
Fail
1425: 697990423684476929
Fail
1426: 697943111201378304
Fail
1427: 697881462549430272
Fail
1428: 697630435728322560
Fail
1429: 697616773278015490
Fail
1430: 697596423848730625
Fail
1431: 697575480820686848
Fail
1432: 697516214579523584
Fail
1433: 697482927769255936
Fail
1434: 697463031882764288
Fail
1435: 697270446429966336
Fail
1436: 697259378236399616
Fail
1437: 697255105972801536
Fail
1438: 697242256848379904
Fail
1439: 696900204696625153
Fail
1440: 696894894812565505
Fail
1441:

Fail
1683: 681891461017812993
Fail
1684: 681694085539872773
Fail
1685: 681679526984871937
Fail
1686: 681654059175129088
Fail
1687: 681610798867845120
Fail
1688: 681579835668455424
Fail
1689: 681523177663676416
Fail
1690: 681340665377193984
Fail
1691: 681339448655802368
Fail
1692: 681320187870711809
Fail
1693: 681302363064414209
Fail
1694: 681297372102656000
Fail
1695: 681281657291280384
Fail
1696: 681261549936340994
Fail
1697: 681242418453299201
Fail
1698: 681231109724700672
Fail
1699: 681193455364796417
Fail
1700: 680970795137544192
Fail
1701: 680959110691590145
Fail
1702: 680940246314430465
Fail
1703: 680934982542561280
Fail
1704: 680913438424612864
Fail
1705: 680889648562991104
Fail
1706: 680836378243002368
Fail
1707: 680805554198020098
Fail
1708: 680801747103793152
Fail
1709: 680798457301471234
Fail
1710: 680609293079592961
Fail
1711: 680583894916304897
Fail
1712: 680497766108381184
Fail
1713: 680494726643068929
Fail
1714: 680473011644985345
Fail
1715: 680440374763077632
Fail
1716:

Fail
1957: 673612854080196609
Fail
1958: 673583129559498752
Fail
1959: 673580926094458881
Fail
1960: 673576835670777856
Fail
1961: 673363615379013632
Fail
1962: 673359818736984064
Fail
1963: 673355879178194945
Fail
1964: 673352124999274496
Fail
1965: 673350198937153538
Fail
1966: 673345638550134785
Fail
1967: 673343217010679808
Fail
1968: 673342308415348736
Fail
1969: 673320132811366400
Fail
1970: 673317986296586240
Fail
1971: 673295268553605120
Fail
1972: 673270968295534593
Fail
1973: 673240798075449344
Fail
1974: 673213039743795200
Fail
1975: 673148804208660480
Fail
1976: 672997845381865473
Fail
1977: 672995267319328768
Fail
1978: 672988786805112832
Fail
1979: 672984142909456390
Fail
1980: 672980819271634944
Fail
1981: 672975131468300288
Fail
1982: 672970152493887488
Fail
1983: 672968025906282496
Fail
1984: 672964561327235073
Fail
1985: 672902681409806336
Fail
1986: 672898206762672129
Fail
1987: 672884426393653248
Fail
1988: 672877615439593473
Fail
1989: 672834301050937345
Fail
1990:

Fail
2231: 668237644992782336
Fail
2232: 668226093875376128
Fail
2233: 668221241640230912
Fail
2234: 668204964695683073
Fail
2235: 668190681446379520
Fail
2236: 668171859951755264
Fail
2237: 668154635664932864
Fail
2238: 668142349051129856
Fail
2239: 668113020489474048
Fail
2240: 667937095915278337
Fail
2241: 667924896115245057
Fail
2242: 667915453470232577
Fail
2243: 667911425562669056
Fail
2244: 667902449697558528
Fail
2245: 667886921285246976
Fail
2246: 667885044254572545
Fail
2247: 667878741721415682
Fail
2248: 667873844930215936
Fail
2249: 667866724293877760
Fail
2250: 667861340749471744
Fail
2251: 667832474953625600
Fail
2252: 667806454573760512
Fail
2253: 667801013445750784
Fail
2254: 667793409583771648
Fail
2255: 667782464991965184
Fail
2256: 667773195014021121
Fail
2257: 667766675769573376
Fail
2258: 667728196545200128
Fail
2259: 667724302356258817
Fail
2260: 667550904950915073
Fail
2261: 667550882905632768
Fail
2262: 667549055577362432
Fail
2263: 667546741521195010
Fail
2264:

The code above has to be ran ONCE!!!
The JSON text file was downloaded from Udacity due to API keys issues.

In [9]:
# Create dataframe columns id, retweet_count, and favorite_count with tweer_json.text 

twt_id = []
faves = []
rts = []
with open('tweet_json.txt', mode = 'r') as f:
     for line in f.readlines():
            tweet_data = json.loads(line)
            twt_id.append(tweet_data['id'])
            faves.append(tweet_data['favorite_count'])
            rts.append(tweet_data['retweet_count'])
        
df_tweet = pd.DataFrame({'tweet_id':twt_id, 'favorite_count':faves, 'retweet_count':rts})

In [10]:
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
favorite_count    2354 non-null int64
retweet_count     2354 non-null int64
tweet_id          2354 non-null int64
dtypes: int64(3)
memory usage: 55.2 KB


In [11]:
df_tweet.head()

,favorite_count,retweet_count,tweet_id
0,39467,8853,892420643555336193
1,33819,6514,892177421306343426
2,25461,4328,891815181378084864
3,42908,8964,891689557279858688
4,41048,9774,891327558926688256


In [12]:
# make a copy of df_tweet_json
df_twee = df_tweet.copy()

# Assessing data

## Quality issue dimensions are:
### Completeness:<br>
Do we have all of the records that we need?<br>
Do we have missing records or not?<br>
Are there specific rows, columns, or cells missing? <br>
### Validity:<br>
We have the records, but they’re not valid, i.e., they don’t conform to a defined schema. A schema is a defined set of rules for data. These rules can be real-world constraints (e.g. negative height is impossible) and table-specific constraints (e.g. unique key constraints in tables).
### Accuracy:<br>
Inaccurate data is wrong data that is valid. It adheres to the defined schema, but it is still incorrect. Example: a patient’s weight that is 5 lbs too heavy because the scale was faulty.<br>
### Consistency:<br>
Inconsistent data is both valid and accurate, but there are multiple correct ways of referring to the same thing. Consistency, i.e., a standard format, in columns that represent the same data across tables and/or within tables is desired.

##  Tidy Data requirements:<br>
Each variable forms a column <br>
Each observation forms a row <br>
Each type of observational unit forms a table

In [13]:
# Check data types are compatible and columns are not missing entries.
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [14]:
df_twitter.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [15]:
df_twitter.nlargest(10, ['rating_numerator'])

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
979,749981277374128128,NaN,NaN,2016-07-04 15:00:45 +0000,"<a href=""https://about.twitter.com/products/tw...",This is Atticus. He's quite simply America af....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/749981277...,1776,10,Atticus,None,None,None,None
313,835246439529840640,8.352460e+17,2.625958e+07,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
189,855860136149123072,8.558585e+17,1.361572e+07,2017-04-22 19:05:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@s8n You tried very hard to portray this good ...,NaN,NaN,NaN,NaN,666,10,None,None,None,None,None
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None
2074,670842764863651840,NaN,NaN,2015-11-29 05:52:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",After so many requests... here you go.\n\nGood...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670842764...,420,10,None,None,None,None,None
1120,731156023742988288,NaN,NaN,2016-05-13 16:15:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to this unbelievably well behaved sq...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/731156023...,204,170,this,None,None,None,None
290,838150277551247360,8.381455e+17,2.195506e+07,2017-03-04 22:12:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@markhoppus 182/10,NaN,NaN,NaN,NaN,182,10,None,None,None,None,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244...,165,150,None,None,None,None,None
1779,677716515794329600,NaN,NaN,2015-12-18 05:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",IT'S PUPPERGEDDON. Total of 144/120 ...I think...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/677716515...,144,120,None,None,None,None,None
1634,684225744407494656,6.842229e+17,4.196984e+09,2016-01-05 04:11:44 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Two sneaky puppers were not initially seen, mo...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/684225744...,143,130,None,None,None,None,None


In [16]:
df_img.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [17]:
sum(df_img.img_num != 1)

295

In [18]:
df_img.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.0+ KB


In [19]:
df_twee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
favorite_count    2354 non-null int64
retweet_count     2354 non-null int64
tweet_id          2354 non-null int64
dtypes: int64(3)
memory usage: 55.2 KB


In [20]:
df_twitter.isnull().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [21]:
df_img.isnull().sum()

tweet_id    0
jpg_url     0
img_num     0
p1          0
p1_conf     0
p1_dog      0
p2          0
p2_conf     0
p2_dog      0
p3          0
p3_conf     0
p3_dog      0
dtype: int64

In [22]:
df_twee.isnull().sum()

favorite_count    0
retweet_count     0
tweet_id          0
dtype: int64

In [23]:
df_twitter.duplicated().sum()

0

In [24]:
df_img.duplicated().sum()

0

In [25]:
df_twee.duplicated().sum()

0

In [26]:
# Detect entries where there are more than one dog stage.
df_twitter.loc[(df_twitter[['doggo', 'floofer', 'pupper', 'puppo']] != 'None').sum(axis=1) > 1]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
191,855851453814013952,NaN,NaN,2017-04-22 18:31:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a puppo participating in the #ScienceMa...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/855851453...,13,10,None,doggo,None,None,puppo
200,854010172552949760,NaN,NaN,2017-04-17 16:34:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...","At first I thought this was a shy doggo, but i...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/854010172...,11,10,None,doggo,floofer,None,None
460,817777686764523521,NaN,NaN,2017-01-07 16:59:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Dido. She's playing the lead role in ""...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/817777686...,13,10,Dido,doggo,None,pupper,None
531,808106460588765185,NaN,NaN,2016-12-12 00:29:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have Burke (pupper) and Dexter (doggo)...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/808106460...,12,10,None,doggo,None,pupper,None
565,802265048156610565,7.331095e+17,4.196984e+09,2016-11-25 21:37:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Like doggo, like pupper version 2. Both 11/10 ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/802265048...,11,10,None,doggo,None,pupper,None
575,801115127852503040,NaN,NaN,2016-11-22 17:28:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bones. He's being haunted by another d...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/801115127...,12,10,Bones,doggo,None,pupper,None
705,785639753186217984,NaN,NaN,2016-10-11 00:34:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Pinot. He's a sophisticated doggo. You...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/785639753...,10,10,Pinot,doggo,None,pupper,None
733,781308096455073793,NaN,NaN,2016-09-29 01:42:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...","Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",NaN,NaN,NaN,https://vine.co/v/5rgu2Law2ut,12,10,None,doggo,None,pupper,None
778,775898661951791106,NaN,NaN,2016-09-14 03:27:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: Like father (doggo), like son (...",7.331095e+17,4.196984e+09,2016-05-19 01:38:16 +0000,https://twitter.com/dog_rates/status/733109485...,12,10,None,doggo,None,pupper,None
822,770093767776997377,NaN,NaN,2016-08-29 03:00:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is just downright precious...,7.410673e+17,4.196984e+09,2016-06-10 00:39:48 +0000,https://twitter.com/dog_rates/status/741067306...,12,10,just,doggo,None,pupper,None


In [27]:
df_img.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [28]:
df_twee.describe()

,favorite_count,retweet_count,tweet_id
count,2354.000000,2354.000000,2.354000e+03
mean,8080.968564,3164.797366,7.426978e+17
std,11814.771334,5284.770364,6.852812e+16
min,0.000000,0.000000,6.660209e+17
25%,1415.000000,624.500000,6.783975e+17
50%,3603.500000,1473.500000,7.194596e+17
75%,10122.250000,3652.000000,7.993058e+17
max,132810.000000,79515.000000,8.924206e+17


In [29]:
# Make sure numerators are consistent
df_twitter.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [30]:
df_twitter.rating_numerator.unique()

array([  13,   12,   14,    5,   17,   11,   10,  420,  666,    6,   15,
        182,  960,    0,   75,    7,   84,    9,   24,    8,    1,   27,
          3,    4,  165, 1776,  204,   50,   99,   80,   45,   60,   44,
        143,  121,   20,   26,    2,  144,   88], dtype=int64)

In [32]:
# Make sure denominators are consistent
df_twitter.rating_denominator.unique()

array([ 10,   0,  15,  70,   7,  11, 150, 170,  20,  50,  90,  80,  40,
       130, 110,  16, 120,   2], dtype=int64)

In [33]:
sum(df_twitter.doggo == 'None')

2259

In [34]:
sum(df_twitter.floofer =='None')

2346

In [35]:
sum(df_twitter.pupper == 'None')

2099

In [36]:
sum(df_twitter.puppo == 'None')

2326

In [37]:
sum(df_twitter.name == 'None')

745

In [38]:
sum(df_twitter.name == 'a')

55

In [41]:
# Check if there are more retweets than favorites for a tweet (as this is unusual)
df_twee[df_twee['retweet_count'] > df_twee['favorite_count']]

,favorite_count,retweet_count,tweet_id
31,0,108,886054160059072513
35,0,19297,885311592912609280
67,0,7181,879130579576475649
72,0,1349,878404777348136964
73,0,6965,878316110768087041
77,0,83,877611172832227328
90,0,15546,874434818259525634
94,0,12518,873697596434513921
96,0,1667,873337748698140672
100,0,31,872668790621863937


In [42]:
df_img.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


## Quality Issues
+ Columns relevant to retweets need to be dropped( we only want original tweet with images) <br>
+ Replies related columns are not really needed. <br>
+ Column 'name' does not follow the rules
+ Convert tweet_id to string.<br>
+ Convert timestamp to datetime. <br>
+ Rating numerator value equal 1776 is an outlier. <br>
+ Delete all rating numerator value superior at 20.
+ Numerators equal at 20 or less are acceptable.   
+ Columns expanded URLs is unnecessary in df_twitter.
+ Capitalize all words founds in columns p1, p2 and p3.

## Tidiness Issues
+ Merge all dataframes in one by tweet_id.
+ Doggo, floofer, pupper, puppo are one variable spread across different columns
+ rating_numerator and rating_denominator can be combined into one column 
+ It would be useful to split the timestamp column in columns: date, year, month, day of the week, time and hour.

# Cleaning data
Now, it's time to clean our dataset based on the issues we found in the assessment part of this wrangling project.
We have already made a copy of all 3 datasets:
df_twitter = df_twitter_archive.copy()<br>
df_img = df_image.copy()<br>
df_twee = df_tweet.copy() 

In [43]:
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [44]:
df_img.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.0+ KB


In [46]:
df_twee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
favorite_count    2354 non-null int64
retweet_count     2354 non-null int64
tweet_id          2354 non-null int64
dtypes: int64(3)
memory usage: 55.2 KB


Tidiness issue

+ Define<br>
First, we are going to merge our three datasets. We will set the how parameter to 'inner'; that way we only deal with tweets that are in both tables. After completing this task, we will work with the dataset df_master.

+ Codes

In [47]:
df_archive = pd.merge(df_twitter,df_twee, on = ['tweet_id'], how = 'inner')

In [48]:
df_master = pd.merge(df_archive,df_img, on = ['tweet_id'], how = 'inner')

+ Test

In [143]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2073 entries, 0 to 2072
Data columns (total 30 columns):
tweet_id                      2073 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     2073 non-null object
source                        2073 non-null object
text                          2073 non-null object
retweeted_status_id           79 non-null float64
retweeted_status_user_id      79 non-null float64
retweeted_status_timestamp    79 non-null object
expanded_urls                 2073 non-null object
rating_numerator              2073 non-null int64
rating_denominator            2073 non-null int64
name                          2073 non-null object
doggo                         2073 non-null object
floofer                       2073 non-null object
pupper                        2073 non-null object
puppo                         2073 non-null object
favorite_count                2073 

Quality issue 

+ Define<br>
Drop columns that we dont need in our analysis

+ Code

In [144]:
df_master_arch = df_master.drop(['jpg_url', 'img_num', 'name', 'source', 'text', 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp','expanded_urls'], axis=1)

+ Test

In [145]:
df_master_arch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2073 entries, 0 to 2072
Data columns (total 19 columns):
tweet_id              2073 non-null int64
timestamp             2073 non-null object
rating_numerator      2073 non-null int64
rating_denominator    2073 non-null int64
doggo                 2073 non-null object
floofer               2073 non-null object
pupper                2073 non-null object
puppo                 2073 non-null object
favorite_count        2073 non-null int64
retweet_count         2073 non-null int64
p1                    2073 non-null object
p1_conf               2073 non-null float64
p1_dog                2073 non-null bool
p2                    2073 non-null object
p2_conf               2073 non-null float64
p2_dog                2073 non-null bool
p3                    2073 non-null object
p3_conf               2073 non-null float64
p3_dog                2073 non-null bool
dtypes: bool(3), float64(3), int64(5), object(8)
memory usage: 361.4+ KB


Quality issue 

+ Define<br>
Erroneous datatype for tweet_id<br>
Convert tweet_id to string.

+ Code

In [205]:
df_master_arch['tweet_id'] = df_master_arch['tweet_id'].astype('str')

+ Test

In [206]:
df_master_arch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2054 entries, 0 to 2072
Data columns (total 20 columns):
tweet_id              2054 non-null object
timestamp             2054 non-null object
rating_numerator      2054 non-null float64
rating_denominator    2054 non-null float64
doggo                 2054 non-null object
floofer               2054 non-null object
pupper                2054 non-null object
puppo                 2054 non-null object
favorite_count        2054 non-null int64
retweet_count         2054 non-null int64
p1                    2054 non-null object
p1_conf               2054 non-null float64
p1_dog                2054 non-null bool
p2                    2054 non-null object
p2_conf               2054 non-null float64
p2_dog                2054 non-null bool
p3                    2054 non-null object
p3_conf               2054 non-null float64
p3_dog                2054 non-null bool
rating                2054 non-null float64
dtypes: bool(3), float64(6), int64(

Quality issue 

+ Define <br>
Erroneous rating_nominator and rating_denominator values. We believe that the value 20 or less are accpetable numbers.
Drop entries where rating numerator are superior at 20. 

+ Code

In [207]:
df_master_arch.drop(df_master_arch[df_master_arch['rating_numerator'] > 20].index, inplace= True)

+ Test

In [208]:
# check for the max value for rating numerator
df_master_arch.rating_numerator.max()

15.0

In [209]:
# check for the max value for rating denominator
df_master_arch.rating_denominator.max()

20.0

Quality issue

+ Define <br>
Make sure that numerator and denominator are in compatible format, since we are going to create a column 
ration = ration_numerator/ration_denominator
Convertion  them to float data type

+ Code

In [210]:
df_master_arch['rating_numerator'] = df_master_arch['rating_numerator'].astype(str).astype(float)

In [211]:
df_master_arch['rating_denominator'] = df_master_arch['rating_denominator'].astype(str).astype(float)

+ Test

In [212]:
df_master_arch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2054 entries, 0 to 2072
Data columns (total 20 columns):
tweet_id              2054 non-null object
timestamp             2054 non-null object
rating_numerator      2054 non-null float64
rating_denominator    2054 non-null float64
doggo                 2054 non-null object
floofer               2054 non-null object
pupper                2054 non-null object
puppo                 2054 non-null object
favorite_count        2054 non-null int64
retweet_count         2054 non-null int64
p1                    2054 non-null object
p1_conf               2054 non-null float64
p1_dog                2054 non-null bool
p2                    2054 non-null object
p2_conf               2054 non-null float64
p2_dog                2054 non-null bool
p3                    2054 non-null object
p3_conf               2054 non-null float64
p3_dog                2054 non-null bool
rating                2054 non-null float64
dtypes: bool(3), float64(6), int64(

Tidiness issue

+ Define<br>
Rating_numerator and rating_denominator can be combined into one column 

+ Code

In [213]:
df_master_arch['rating']= df_master_arch['rating_numerator'] / df_master_arch['rating_denominator']

+ Test

In [214]:
df_master_arch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2054 entries, 0 to 2072
Data columns (total 20 columns):
tweet_id              2054 non-null object
timestamp             2054 non-null object
rating_numerator      2054 non-null float64
rating_denominator    2054 non-null float64
doggo                 2054 non-null object
floofer               2054 non-null object
pupper                2054 non-null object
puppo                 2054 non-null object
favorite_count        2054 non-null int64
retweet_count         2054 non-null int64
p1                    2054 non-null object
p1_conf               2054 non-null float64
p1_dog                2054 non-null bool
p2                    2054 non-null object
p2_conf               2054 non-null float64
p2_dog                2054 non-null bool
p3                    2054 non-null object
p3_conf               2054 non-null float64
p3_dog                2054 non-null bool
rating                2054 non-null float64
dtypes: bool(3), float64(6), int64(

Quality issue

+ Define<br>
Drop columns rating_numerator and rating denominator

In [215]:
df_master_arch1 = df_master_arch.drop(['rating_numerator', 'rating_denominator'], axis = 1)

 + Test

In [216]:

df_master_arch1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2054 entries, 0 to 2072
Data columns (total 18 columns):
tweet_id          2054 non-null object
timestamp         2054 non-null object
doggo             2054 non-null object
floofer           2054 non-null object
pupper            2054 non-null object
puppo             2054 non-null object
favorite_count    2054 non-null int64
retweet_count     2054 non-null int64
p1                2054 non-null object
p1_conf           2054 non-null float64
p1_dog            2054 non-null bool
p2                2054 non-null object
p2_conf           2054 non-null float64
p2_dog            2054 non-null bool
p3                2054 non-null object
p3_conf           2054 non-null float64
p3_dog            2054 non-null bool
rating            2054 non-null float64
dtypes: bool(3), float64(4), int64(2), object(9)
memory usage: 262.8+ KB


Quality issue

+ Define<br>
Convert timestamp to datetime 

In [217]:
df_master_arch1['timestamp'] = pd.DatetimeIndex(df_master_arch1['timestamp'])

+ Test

In [218]:
df_master_arch1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2054 entries, 0 to 2072
Data columns (total 18 columns):
tweet_id          2054 non-null object
timestamp         2054 non-null datetime64[ns, UTC]
doggo             2054 non-null object
floofer           2054 non-null object
pupper            2054 non-null object
puppo             2054 non-null object
favorite_count    2054 non-null int64
retweet_count     2054 non-null int64
p1                2054 non-null object
p1_conf           2054 non-null float64
p1_dog            2054 non-null bool
p2                2054 non-null object
p2_conf           2054 non-null float64
p2_dog            2054 non-null bool
p3                2054 non-null object
p3_conf           2054 non-null float64
p3_dog            2054 non-null bool
rating            2054 non-null float64
dtypes: bool(3), datetime64[ns, UTC](1), float64(4), int64(2), object(8)
memory usage: 262.8+ KB


Quality issue

+ Define<br>
Many dog breed names are in lowercase in the p1, the p2 and the p3 columns. 
It's better to capitalize all names across these columns in order to work with 
consistent data.

+ Codes

In [219]:
df_master_arch1['p1'] = df_master_arch1['p1'].str.title()
df_master_arch1['p2'] = df_master_arch1['p2'].str.title()
df_master_arch1['p3'] = df_master_arch1['p3'].str.title()

+ Test

In [220]:
df_master_arch1.loc[:,['p1', 'p2', 'p3']].sample(10)

,p1,p2,p3
709,Norwegian_Elkhound,Malinois,German_Shepherd
820,Kelpie,Irish_Terrier,Dingo
267,Kelpie,German_Short-Haired_Pointer,Staffordshire_Bullterrier
1915,Golden_Retriever,Tibetan_Mastiff,Kuvasz
205,Labrador_Retriever,Chesapeake_Bay_Retriever,Newfoundland
1845,English_Springer,English_Setter,Cocker_Spaniel
108,Eskimo_Dog,Siberian_Husky,Malamute
2071,Redbone,Miniature_Pinscher,Rhodesian_Ridgeback
274,Pembroke,Cardigan,Appenzeller
1910,Cocker_Spaniel,Lakeland_Terrier,Soft-Coated_Wheaten_Terrier


In [221]:
df_master_arch1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2054 entries, 0 to 2072
Data columns (total 18 columns):
tweet_id          2054 non-null object
timestamp         2054 non-null datetime64[ns, UTC]
doggo             2054 non-null object
floofer           2054 non-null object
pupper            2054 non-null object
puppo             2054 non-null object
favorite_count    2054 non-null int64
retweet_count     2054 non-null int64
p1                2054 non-null object
p1_conf           2054 non-null float64
p1_dog            2054 non-null bool
p2                2054 non-null object
p2_conf           2054 non-null float64
p2_dog            2054 non-null bool
p3                2054 non-null object
p3_conf           2054 non-null float64
p3_dog            2054 non-null bool
rating            2054 non-null float64
dtypes: bool(3), datetime64[ns, UTC](1), float64(4), int64(2), object(8)
memory usage: 262.8+ KB


Tidiness issue

+ Define<br>
Four variables (doggo, floofer, pupper, puppo) should be merged in one column

+ Codes

In [222]:

df_master_arch1['doggo'].replace('None', '', inplace=True)
df_master_arch1['floofer'].replace('None', '', inplace=True)
df_master_arch1['pupper'].replace('None', '', inplace=True)
df_master_arch1['puppo'].replace('None', '', inplace=True)

In [223]:
df_master_arch1['stage'] = (df_master_arch1['doggo'] +  df_master_arch1['floofer'] + 
                        df_master_arch1['pupper'] + df_master_arch1['puppo'])

In [224]:

df_master_arch1['stage'].replace('', 'Unidentified', inplace=True)

In [225]:
df_master_arch2= df_master_arch1.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1)

+ Test

In [226]:
# It seems that 11 dogs are identified belonging in two 'stages':
df_master_arch2['stage'].value_counts()

Unidentified    1735
pupper           209
doggo             67
puppo             23
doggopupper       11
floofer            7
doggopuppo         1
doggofloofer       1
Name: stage, dtype: int64

In [227]:
df_master_arch2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2054 entries, 0 to 2072
Data columns (total 15 columns):
tweet_id          2054 non-null object
timestamp         2054 non-null datetime64[ns, UTC]
favorite_count    2054 non-null int64
retweet_count     2054 non-null int64
p1                2054 non-null object
p1_conf           2054 non-null float64
p1_dog            2054 non-null bool
p2                2054 non-null object
p2_conf           2054 non-null float64
p2_dog            2054 non-null bool
p3                2054 non-null object
p3_conf           2054 non-null float64
p3_dog            2054 non-null bool
rating            2054 non-null float64
stage             2054 non-null object
dtypes: bool(3), datetime64[ns, UTC](1), float64(4), int64(2), object(5)
memory usage: 214.6+ KB


Tidiness issue

+ Define<br>
The timestamp is already converted to datetime; now, we can split it to date, time, day of week and hour.

In [228]:
temp = pd.DatetimeIndex(df_master_arch2['timestamp'])

+ Codes

In [229]:
df_master_arch2['Date'] = temp.date

In [230]:
df_master_arch2['Year'] = temp.year

In [231]:
df_master_arch2['Month'] = temp.month

In [232]:
df_master_arch2['Month'].replace(1, 'January', inplace=True)
df_master_arch2['Month'].replace(2, 'February', inplace=True)
df_master_arch2['Month'].replace(3, 'March', inplace=True)
df_master_arch2['Month'].replace(4, 'April', inplace=True)
df_master_arch2['Month'].replace(5, 'May', inplace=True)
df_master_arch2['Month'].replace(6, 'June', inplace=True)
df_master_arch2['Month'].replace(7, 'July', inplace=True)
df_master_arch2['Month'].replace(8, 'August', inplace=True)
df_master_arch2['Month'].replace(9, 'September', inplace=True)
df_master_arch2['Month'].replace(10, 'October', inplace=True)
df_master_arch2['Month'].replace(11, 'November', inplace=True)
df_master_arch2['Month'].replace(12, 'December', inplace=True)

In [233]:
df_master_arch2['Day_of_Week'] = temp.dayofweek

In [234]:
df_master_arch2['Day_of_Week'].replace(0, 'Monday', inplace=True)
df_master_arch2['Day_of_Week'].replace(1, 'Tuesday', inplace=True)
df_master_arch2['Day_of_Week'].replace(2, 'Wednesday', inplace=True)
df_master_arch2['Day_of_Week'].replace(3, 'Thursday', inplace=True)
df_master_arch2['Day_of_Week'].replace(4, 'Friday', inplace=True)
df_master_arch2['Day_of_Week'].replace(5, 'Saturday', inplace=True)
df_master_arch2['Day_of_Week'].replace(6, 'Sunday', inplace=True)

In [235]:
df_master_arch2['Time'] = temp.time

In [236]:
df_master_arch2['Hour'] = temp.hour

In [237]:
df_master_arch3 = df_master_arch2.drop('timestamp', axis=1)

+ Test

In [238]:
df_master_arch3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2054 entries, 0 to 2072
Data columns (total 20 columns):
tweet_id          2054 non-null object
favorite_count    2054 non-null int64
retweet_count     2054 non-null int64
p1                2054 non-null object
p1_conf           2054 non-null float64
p1_dog            2054 non-null bool
p2                2054 non-null object
p2_conf           2054 non-null float64
p2_dog            2054 non-null bool
p3                2054 non-null object
p3_conf           2054 non-null float64
p3_dog            2054 non-null bool
rating            2054 non-null float64
stage             2054 non-null object
Date              2054 non-null object
Year              2054 non-null int64
Month             2054 non-null object
Day_of_Week       2054 non-null object
Time              2054 non-null object
Hour              2054 non-null int64
dtypes: bool(3), float64(4), int64(4), object(9)
memory usage: 294.9+ KB


In [239]:
df_master_arch3.sample(5)

,tweet_id,favorite_count,retweet_count,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,rating,stage,Date,Year,Month,Day_of_Week,Time,Hour
1081,708026248782585858,4799,2166,Malinois,0.786468,True,Chesapeake_Bay_Retriever,0.068979,True,Siamese_Cat,0.029304,False,1.3,Unidentified,2016-03-10,2016,March,Thursday,20:26:26,20
952,726828223124897792,3882,1124,Miniature_Pinscher,0.255327,True,Border_Terrier,0.181279,True,Labrador_Retriever,0.125185,True,1.2,Unidentified,2016-05-01,2016,May,Sunday,17:38:46,17
1792,671115716440031232,1436,842,Malinois,0.406341,True,Kelpie,0.143366,True,Dingo,0.129802,False,0.6,Unidentified,2015-11-29,2015,November,Sunday,23:57:10,23
1955,668142349051129856,592,306,Angora,0.918834,False,Hen,0.037793,False,Wood_Rabbit,0.011015,False,0.2,Unidentified,2015-11-21,2015,November,Saturday,19:02:04,19
1842,670408998013820928,600,249,Ping-Pong_Ball,0.999945,False,Tennis_Ball,0.000018,False,Racket,0.000015,False,1.0,Unidentified,2015-11-28,2015,November,Saturday,01:08:55,1


In [240]:
df_master_arch3.isnull().sum()

tweet_id          0
favorite_count    0
retweet_count     0
p1                0
p1_conf           0
p1_dog            0
p2                0
p2_conf           0
p2_dog            0
p3                0
p3_conf           0
p3_dog            0
rating            0
stage             0
Date              0
Year              0
Month             0
Day_of_Week       0
Time              0
Hour              0
dtype: int64

In [241]:
df_master_arch3.duplicated().sum()

0

Now, we are ready to save our clean dataFrame df_master_arch3 in a csv file

In [242]:
df_master_arch3.to_csv('twitter_archive_master.csv', index=False)

Now, it's time to use the cleaned master Dataframe to analyse and create relevant and interesting visualizations.